In [1]:
from datetime import datetime, timedelta, timezone
import threading
import time 


In [2]:
class Scheduler:
    """
    A class to represent a schedule for a job with a start and end date.
    
    Attributes:
    -----------
    stdate : str or datetime
        The start date and time of the schedule. If not provided, defaults to the current date and time.
    time_zone : str
        The timezone of the schedule. Defaults to the current timezone of the computer.
    
    Methods:
    --------
    add_job():
        Adds a job for running it 
    run_all():
        Runs all jobs with ot without threading
        
    Raises:
    -------
    ValueError:
        If Startdate is less than the current date.
    """

    def __init__(self, threading=False, startdate=None, time_zone=None):
        self.threads = threading
        self.stdate = startdate if startdate else datetime.now().strftime('%Y-%m-%d %H:%M')
        self.stdate = datetime.strptime(self.stdate, '%Y-%m-%d %H:%M')
        self.jobs = []

        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
        else:
            # use the system's local timezone if no offset is provided
            self.time_zone = datetime.now().astimezone().tzinfo

        # checking if startdate is less than the current date
        if self.stdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Startdate cannot be less than the current date")
        
        # counting time for specified timezone 
        startdate_with_timezone = self.stdate.astimezone(self.time_zone)




############################# METHODS #############################
    
    ####### add_job ####### 
    def add_job(self, job):
        # if the job doesn't have a startdate, use the scheduler's startdate
        if job.startdate is None:
            job.startdate = self.stdate
            
        # if the job doesn't have a time_zone, use the scheduler's time_zone
        if job.time_zone is None:
            job.time_zone = self.time_zone
        # adding jobs 
        self.jobs.append(job)
        return self


    ####### run_all ####### 
    def run_all(self):
        
        # running jobs simultaneously when threads is true
        if self.threads:
            threads = []
            for job in self.jobs:
                # calling the Job run method for each thread
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                
                # starting the execution of the thread
                thread.start()
            
            # ensure that run_all doesn't finish until all jobs have completed their execution.
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                job.run()
                
    def run_job(self, job):
        if job.interval:
            while True:
                job.run()
                time.sleep(job.interval)
        else:
            job.run()


In [8]:
# Example Job Class
class Job:
    def __init__(self, startdate =None, time_zone=None):
        # starting date of shcedule
        self.startdate = startdate
        # timezone hour offset
        self.time_zone =time_zone
        # function
        self.func = None
        # name of function
        self.name = None
        # time unit 
        self.unit = None
        self.interval = None
        self.next_run = None

        # setting time_zone 
        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
        if isinstance(self.startdate, str):
            # Converting the string to a datetime object
                self.startdate = datetime.strptime(self.startdate, '%Y-%m-%d %H:%M')
                # Checking if startdate is less than the current date
                if self.startdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
                    raise ValueError("Job startdate cannot be less than the current date")


    ############################# METHODS #############################
    
    def do(self, func, name):
        self.func = func
        self.name = name
        return self
    
    
    @property
    def second(self):
        self.unit = 'second'
        # self.interval = interval
        return self

    def every(self, interval):
        self.unit = 'second'
        self.interval = interval
        return self
    
    def calc_next_run(self, current_time):
        if self.unit == 'second':
            return current_time + timedelta(seconds=self.interval)
    
    # რეალური რანის ფუნქცია მაქვს დასაწერი !!!!!!!!!!!

    def run(self):
        self.next_run = self.calc_next_run(self.startdate)
        # Checking if self.startdate is a string
        if isinstance(self.startdate, str):
            self.startdate = datetime.strptime(self.startdate, '%Y-%m-%d %H:%M')
        # Counting time for specified timezone 
        startdate_with_timezone = self.startdate.astimezone(self.time_zone)
        # searching if there are functions to run 
        if self.func is not None:
            # running and giving response that running is finished
            self.func()   
            print(f'next run {startdate_with_timezone}')
        else:
            # for testing purposes needs changing ofc 
            print('IAMNOOOOOOOOOOOOOOOOOOOOOONEEEEEEEEEEEEEEEEEE')
        time.sleep(5)
        

# Example functions
def report1():
    print("Function report1 is running.")

def report2():
    print("Function report2 is running.")

def report3():
    print("Function report3 is running.")

# Create Job instances and associate functions with them
job1 = Job(startdate='2024-08-15 10:00', time_zone=4).do(report1, 'report1')
job2 = Job(startdate='2024-08-15 10:00', time_zone=2).do(report2, 'report2')
job3 = Job(startdate='2024-08-15 10:00', time_zone=6).do(report3, 'report3')


# Now you can add these jobs to the Scheduler and run them
scheduler = Scheduler(threading=False)
scheduler.add_job(job1)
scheduler.add_job(job2)
scheduler.add_job(job3)
scheduler.run_all()

Function report1 is running.
next run 2024-08-15 10:00:00+04:00
Function report2 is running.
next run 2024-08-15 08:00:00+02:00
Function report3 is running.
next run 2024-08-15 12:00:00+06:00


In [ ]:
# როდესაც ჯობიისთის გადაცემული გაშვების თარიღი 